In [245]:
%matplotlib inline

In [247]:
from __future__ import print_function
import pandas as pd
from sklearn.linear_model import LinearRegression
import re
from datetime import datetime
import pickle
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [61]:
pd.options.display.max_rows = 999

In [177]:
googledf = pd.read_csv("GoogleStevenTimeline.csv")

In [178]:
googledf = googledf.iloc[119:]
googledf.head()

,Category: All categories
2013-11,11
2013-12,6
2014-01,6
2014-02,6
2014-03,7


In [179]:
googledf.drop(googledf.tail(5).index,inplace=True)

In [180]:
googledf.reset_index(inplace=True)

In [181]:
googledf.head()

,index,Category: All categories
0,2013-11,11
1,2013-12,6
2,2014-01,6
3,2014-02,6
4,2014-03,7


In [182]:
googledf = googledf.rename(index=str, columns={"index": 'date', "Category: All categories": "search interest"})

In [183]:
googledf.head()

,date,search interest
0,2013-11,11
1,2013-12,6
2,2014-01,6
3,2014-02,6
4,2014-03,7


In [185]:
episodedf = pd.read_csv("finaleps.csv")
episodedf

,Unnamed: 0,No.overall,Directed by,No. inseason,date,Prod.code,episode,Viewers_millions,Written and storyboarded by
0,0,1,Ian Jones-Quartey (supervising)Kevin Dart (art),1,2013-11-04,1020-003,"""Gem Glow""",1.86,Joe Johnston and Jeff Liu
1,1,2,Ian Jones-Quartey (supervising)Kevin Dart (art),2,2013-11-04,1020-001,"""Laser Light Cannon""",1.86,Kat Morris and Rebecca Sugar
2,2,3,Ian Jones-Quartey (supervising)Kevin Dart (art),3,2013-11-11,1020-002,"""Cheeseburger Backpack""",1.68,Ian Jones-Quartey and Rebecca Sugar
3,3,4,Ian Jones-Quartey (supervising)Kevin Dart (art),4,2013-11-11,1020-004,"""Together Breakfast""",1.68,"Ian Jones-Quartey, Rebecca Sugar, and Paul Vil..."
4,4,5,Ian Jones-Quartey (supervising)Kevin Dart (art),5,2013-11-18,1020-008,"""Frybo""",1.29,Raven M. Molisee and Paul Villeco
5,5,6,Ian Jones-Quartey (supervising)Kevin Dart (art),6,2013-11-25,1020-005,"""Cat Fingers""",1.71,"Kat Morris, Hilary Florido, Ian Jones-Quartey,..."
6,6,7,Ian Jones-Quartey (supervising)Kevin Dart (art),7,2013-12-02,1020-006,"""Bubble Buddies""",1.61,Kat Morris and Aleth Romanillos
7,7,8,Ian Jones-Quartey (supervising)Kevin Dart (art),8,2014-01-13,1020-007,"""Serious Steven""",1.35,Joe Johnston and Jeff Liu
8,8,9,Ian Jones-Quartey (supervising)Kevin Dart (art),9,2014-01-20,1020-011,"""Tiger Millionaire""",1.57,Raven M. Molisee and Paul Villeco
9,9,10,Ian Jones-Quartey (supervising)Kevin Dart and ...,10,2014-01-27,1020-012,"""Steven's Lion""",1.51,Lamar Abrams and Aleth Romanillos


In [201]:
viewers = episodedf[['date', 'Viewers_millions']]
viewers.head()

,date,Viewers_millions
0,2013-11-04,1.86
1,2013-11-04,1.86
2,2013-11-11,1.68
3,2013-11-11,1.68
4,2013-11-18,1.29


In [202]:
viewers

,date,Viewers_millions
0,2013-11-04,1.86
1,2013-11-04,1.86
2,2013-11-11,1.68
3,2013-11-11,1.68
4,2013-11-18,1.29
5,2013-11-25,1.71
6,2013-12-02,1.61
7,2014-01-13,1.35
8,2014-01-20,1.57
9,2014-01-27,1.51


In [203]:
viewers['date'] = pd.to_datetime(viewers['date'])


/Users/flatironschool/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [204]:

viewers["date"] = pd.to_datetime(viewers["date"], format='%Y00%m').apply(lambda x: x.strftime('%Y-%m'))
print(viewers["date"])

0      2013-11
1      2013-11
2      2013-11
3      2013-11
4      2013-11
5      2013-11
6      2013-12
7      2014-01
8      2014-01
9      2014-01
10     2014-02
11     2014-02
12     2014-03
13     2014-03
14     2014-03
15     2014-04
16     2014-04
17     2014-04
18     2014-05
19     2014-08
20     2014-08
21     2014-09
22     2014-09
23     2014-09
24     2014-09
25     2014-09
26     2014-10
27     2014-10
28     2014-10
29     2014-10
30     2014-10
31     2014-11
32     2014-11
33     2014-11
34     2014-12
35     2015-01
36     2015-01
37     2015-01
38     2015-01
39     2015-02
40     2015-02
41     2015-02
42     2015-02
43     2015-03
44     2015-03
45     2015-03
46     2015-04
47     2015-04
48     2015-04
49     2015-03
50     2015-03
51     2015-03
52     2015-03
53     2015-03
54     2015-03
55     2015-04
56     2015-04
57     2015-06
58     2015-06
59     2015-06
60     2015-06
61     2015-06
62     2015-07
63     2015-07
64     2015-07
65     2015-07
66     201

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [205]:
viewers

,date,Viewers_millions
0,2013-11,1.86
1,2013-11,1.86
2,2013-11,1.68
3,2013-11,1.68
4,2013-11,1.29
5,2013-11,1.71
6,2013-12,1.61
7,2014-01,1.35
8,2014-01,1.57
9,2014-01,1.51


In [213]:
viewers = viewers.groupby(['date']).mean()

In [214]:
viewers

,index,Viewers_millions
date,,
2013-11,2.5,1.680000
2013-12,6.0,1.610000
2014-01,8.0,1.476667
2014-02,10.5,1.530000
2014-03,13.0,1.486667
2014-04,16.0,1.370000
2014-05,18.0,1.550000
2014-08,19.5,1.740000
2014-09,23.0,2.124000


In [219]:
viewers = viewers.drop(columns=['index'])

In [221]:
viewers.reset_index(inplace=True)

In [222]:
viewers

,date,Viewers_millions
0,2013-11,1.680000
1,2013-12,1.610000
2,2014-01,1.476667
3,2014-02,1.530000
4,2014-03,1.486667
5,2014-04,1.370000
6,2014-05,1.550000
7,2014-08,1.740000
8,2014-09,2.124000
9,2014-10,1.886000


In [210]:
googledf

,date,search interest
0,2013-11,11
1,2013-12,6
2,2014-01,6
3,2014-02,6
4,2014-03,7
5,2014-04,6
6,2014-05,5
7,2014-06,3
8,2014-07,4
9,2014-08,6


In [234]:
test = pd.merge(googledf,
                 viewers[['date', 'Viewers_millions']],
                 on='date')
test.head()

,date,search interest,Viewers_millions
0,2013-11,11,1.680000
1,2013-12,6,1.610000
2,2014-01,6,1.476667
3,2014-02,6,1.530000
4,2014-03,7,1.486667


In [235]:
test

,date,search interest,Viewers_millions
0,2013-11,11,1.680000
1,2013-12,6,1.610000
2,2014-01,6,1.476667
3,2014-02,6,1.530000
4,2014-03,7,1.486667
5,2014-04,6,1.370000
6,2014-05,5,1.550000
7,2014-08,6,1.740000
8,2014-09,10,2.124000
9,2014-10,9,1.886000


In [240]:
scaler = MinMaxScaler()
print(scaler.fit(test[['search interest', 'Viewers_millions']]))

MinMaxScaler(copy=True, feature_range=(0, 1))


/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float64, object were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [241]:
print(scaler.data_max_)

[100.      2.124]


In [256]:
df = pd.DataFrame(scaler.transform(test[['search interest', 'Viewers_millions']]))

In [257]:
df

,0,1
0,0.063158,0.714745
1,0.010526,0.669772
2,0.010526,0.584110
3,0.010526,0.618375
4,0.021053,0.590534
5,0.010526,0.515580
6,0.000000,0.631224
7,0.010526,0.753293
8,0.052632,1.000000
9,0.042105,0.847093


In [258]:
x = df[0]
y = df[1]

In [259]:
model = sm.OLS(y, x)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      1   R-squared:                       0.432
Model:                            OLS   Adj. R-squared:                  0.417
Method:                 Least Squares   F-statistic:                     28.88
Date:                Mon, 10 Jun 2019   Prob (F-statistic):           4.10e-06
Time:                        16:02:21   Log-Likelihood:                -24.313
No. Observations:                  39   AIC:                             50.63
Df Residuals:                      38   BIC:                             52.29
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             1.0858      0.202      5.374      0.0